In [1]:
import pandas as pd
import numpy as np

# Pretty display for notebooks
%matplotlib inline

admins_f = 'flatiron_qs_orders_admins_admins.csv'
demos_f = 'flatiron_qs_orders_admins_demographs.csv'
orders_f = 'flatiron_qs_orders_admins_orders.csv'
patients_f = 'flatiron_qs_orders_admins_patients.csv'
practices_f = 'flatiron_qs_orders_admins_practices.csv'

#admins_df = pd.rea

## Orders

In [18]:
orders_df = pd.read_csv(orders_f)
print orders_df.shape
orders_df.head()

(304, 4)


,patient_id,external_patient_id,order_id,order_date
0,h9993d,70,w187808812917e,13-Dec-2012
1,h9993d,70,r90118525421e,7-Aug-2015
2,h9993d,70,h84358125584w,30-Sep-2015
3,h9993d,70,o81913333209g,8-Jan-2014
4,h9993d,70,e76139113344l,11-Oct-2012


Medication **orders** 
* **patient_id** :  patient identifier for internal purposes
*  **external_patient_id** :  patient identifier for external (client) purposes
*  **order_id** :   unique identifier of each medication ordered by a health provider
*  **order_date** :  date in which medication was ordered by a health provider

## Administrations

In [19]:
admins_df = pd.read_csv(admins_f)
print admins_df.shape
admins_df.head()

(304, 5)


,patient_id,external_patient_id,order_id,administered_date,drug_name
0,h9993d,70,w187808812917e,13-Dec-2012,bevacizumab
1,h9993d,70,r90118525421e,7-Aug-2015,nivolumab
2,h9993d,70,h84358125584w,30-Sep-2015,bevacizumab
3,h9993d,70,o81913333209g,8-Jan-2014,nivolumab
4,h9993d,70,e76139113344l,11-Oct-2012,erlotinib hcl


Medication **administrations**

 * **patient_id** :  patient identifier for internal purposes
 * **external_patient_id** :   patient identifier for external (client) purposes
 * **order_id** :  unique identifier of each medication ordered for a patient by a health provider
 * **administered_date** :  date in which ordered medication was administered
 * **drug_name** :   drug administered to patient

## Demographics

In [41]:
demos_df = pd.read_csv(demos_f)
print demos_df.shape
demos_df.head()

(101, 4)


,patient_id,gender,age,race
0,h9993d,female,69.0,NON_WHITE
1,w9986a,female,46.0,WHITE
2,n9925d,female,84.0,NON_WHITE
3,a9913g,female,14.0,NON_WHITE
4,t9815h,female,97.0,WHITE


Patient **demographics**
*  **patient_id**:   patient identifier for internal purposes
*  **gender** :  gender of patient
*  **age** :  age of patient at the time of diagnosis
*  **race**:   racial group classification of patient

## Question 3
* Using the following risk stratification rules, please summarize the number of high,
medium, and low risk patients.
* HighRisk   = Female; any age; NON_WHITE  OR  Male; >= 70; NON_WHITE
* MediumRisk = Female; >=75, WHITE  OR  Male; <70; NON_WHITE
* LowRisk    = Female; < 75, WHITE  OR  Male; any­age; WHITE

In [42]:
print demos_df['gender'].value_counts()
print demos_df['race'].value_counts()

female     50
male       43
unknown     6
Name: gender, dtype: int64
WHITE        57
NON_WHITE    42
Name: race, dtype: int64


In [43]:
demos_df[demos_df['gender']=='unknown']

,patient_id,gender,age,race
25,w7942e,unknown,100.0,WHITE
33,z7390g,unknown,84.0,NON_WHITE
35,g7173d,unknown,87.0,NON_WHITE
40,a6991u,unknown,95.0,NON_WHITE
54,g5736q,unknown,62.0,NON_WHITE
76,w3752a,unknown,7.0,WHITE


of the 6 listed above, we can assess the risk of 4 out of the six given the criteria above with the following rules:
*    HighRisk : race == 'NON_WHITE' and age > 70
*    LowRisk : race == 'WHITE' and age < 75

In [45]:
def stratify_risk(row):
    #demo_row is a row from the demographics data. It consists of the following columns:
    # patient_id, gender, age, race
    if (row.gender=='female' and row.race=='NON_WHITE') or (row.gender=='male' and row.age>=70 and row.race=='NON_WHITE'):
            return 'HighRisk'
    if (row.gender=='female' and row.age>=75 and row.race=='WHITE') or (row.gender=='male' and row.age<70 and row.race=='NON_WHITE'):
            return 'MediumRisk'
    if (row.gender=='female' and row.age<75 and row.race=='WHITE') or (row.gender=='male' and row.race=='WHITE'):
            return 'LowRisk'
    if (row.race=='NON_WHITE' and row.age>70):
            return 'HighRisk'
    if (row.race=='WHITE' and row.age < 75):
            return 'LowRisk'

demos_df['risk']=demos_df.apply(stratify_risk,axis=1)
demos_df.head()

,patient_id,gender,age,race,risk
0,h9993d,female,69.0,NON_WHITE,HighRisk
1,w9986a,female,46.0,WHITE,LowRisk
2,n9925d,female,84.0,NON_WHITE,HighRisk
3,a9913g,female,14.0,NON_WHITE,HighRisk
4,t9815h,female,97.0,WHITE,MediumRisk


In [46]:
demos_df = demos_df.join(pd.get_dummies(demos_df['gender'], prefix = 'gender'))
demos_df = demos_df.join(pd.get_dummies(demos_df['race'], prefix='race'))
demos_df.head()

,patient_id,gender,age,race,risk,gender_female,gender_male,gender_unknown,race_NON_WHITE,race_WHITE
0,h9993d,female,69.0,NON_WHITE,HighRisk,1.0,0.0,0.0,1.0,0.0
1,w9986a,female,46.0,WHITE,LowRisk,1.0,0.0,0.0,0.0,1.0
2,n9925d,female,84.0,NON_WHITE,HighRisk,1.0,0.0,0.0,1.0,0.0
3,a9913g,female,14.0,NON_WHITE,HighRisk,1.0,0.0,0.0,1.0,0.0
4,t9815h,female,97.0,WHITE,MediumRisk,1.0,0.0,0.0,0.0,1.0


In [47]:
demos_df.groupby('risk').describe()

age  gender_female  gender_male  gender_unknown  \
risk                                                                      
HighRisk   count  31.000000      31.000000    31.000000       31.000000   
           mean   64.967742       0.709677     0.193548        0.096774   
           std    29.460124       0.461414     0.401610        0.300537   
           min     9.000000       0.000000     0.000000        0.000000   
           25%    38.500000       0.000000     0.000000        0.000000   
           50%    75.000000       1.000000     0.000000        0.000000   
           75%    87.000000       1.000000     0.000000        0.000000   
           max    98.000000       1.000000     1.000000        1.000000   
LowRisk    count  47.000000      47.000000    47.000000       47.000000   
           mean   44.234043       0.404255     0.574468        0.021277   
           std    28.511792       0.496053     0.499769        0.145865   
           min     3.000000       0.000000     0.000000        0.000000   
           25%    21.500000       0.000000     0.000000        0.000000   
           50%    38.000000       0.000000     1.000000        0.000000   
           75%    66.000000       1.000000     1.000000        0.000000   
           max    98.000000       1.000000     1.000000        1.000000   
MediumRisk count  19.000000      19.000000    19.000000       19.000000   
           mean   56.263158       0.473684     0.526316        0.000000   
           std    32.756411       0.512989     0.512989        0.000000   
           min     6.000000       0.000000     0.000000        0.000000   
           25%    30.000000       0.000000     0.000000        0.000000   
           50%    62.000000       0.000000     1.000000        0.000000   
           75%    83.500000       1.000000     1.000000        0.000000   
           max    99.000000       1.000000     1.000000        0.000000   

                  race_NON_WHITE  race_WHITE  
risk                                          
HighRisk   count       31.000000   31.000000  
           mean         1.000000    0.000000  
           std          0.000000    0.000000  
           min          1.000000    0.000000  
           25%          1.000000    0.000000  
           50%          1.000000    0.000000  
           75%          1.000000    0.000000  
           max          1.000000    0.000000  
LowRisk    count       47.000000   47.000000  
           mean         0.000000    1.000000  
           std          0.000000    0.000000  
           min          0.000000    1.000000  
           25%          0.000000    1.000000  
           50%          0.000000    1.000000  
           75%          0.000000    1.000000  
           max          0.000000    1.000000  
MediumRisk count       19.000000   19.000000  
           mean         0.526316    0.473684  
           std          0.512989    0.512989  
           min          0.000000    0.000000  
           25%          0.000000    0.000000  
           50%          1.000000    0.000000  
           75%          1.000000    1.000000  
           max          1.000000    1.000000

## Patients

In [21]:
patients_df = pd.read_csv(patients_f)
print patients_df.shape
patients_df.head()

(102, 6)


,patient_id,external_patient_id,external_practice_id,internal_practice_id,diagnosis_date,advanced_diagnosis_date
0,h9993d,70,QZ1,9,29-Mar-1992,NaN
1,w9986a,46,ZE4G,4,15-Feb-2012,3-Jul-2012
2,n9925d,15,FUM35,5,5-Jul-1962,18-May-2009
3,a9913g,6,DSVN67,7,16-Oct-1930,14-Sep-1970
4,a9913g,83,DSVN67,7,16-Oct-1930,14-Sep-1970


**Patients**
*  **patient_id**:   patient identifier for internal purposes
*  **external_patient_id** :   patient identifier for external purposes
*  **external_practice_id** :  practice identifier for external purposes
*  **internal_practice_id** :   practice (site at which patient receives oncology care) identifier for Internal purposes
*  **diagnosis_date** :  date on which patient was initially diagnosed
*  **advanced_diagnosis_date** :   date on which patient was diagnosed as advanced

## Practices

In [22]:
practices_df = pd.read_csv(practices_f)
print practices_df.shape
practices_df.head()

(10, 5)


,practice_id,external_practice_id,emr_system,practice_type,active
0,QZ1,9,OncoE,community,active
1,ZE4G,4,OncoA,community,inactive
2,FUM35,5,OncoA,academic,active
3,DSVN67,7,OncoE,community,inactive
4,FZAB,1,OncoB,community,active


**Practices**
*  **practice_id** :   client identifier for internal purposes
*  **external_practice_id** :  client identifier for external (client) purposes
*  **emr_system** :  software product used by the client (OncoE = OncoEMR; OncoA = OncoAnalytics; OncoB = OncoBilling)
*  **practice_type** :  classification of practice into either community or academic bucket
*  **active** :   status of client

In [9]:
print admins_df['drug_name'].value_counts()
print admins_df['external_patient_id'].value_counts()
ad

nivolumab        109
bevacizumab      104
erlotinib hcl     63
axitinib          28
Name: drug_name, dtype: int64
2      10
50      9
95      8
59      7
70      7
4       6
73      6
75      6
17      5
68      5
71      5
36      5
29      5
54      5
52      5
96      5
9       5
16      4
14      4
49      4
58      4
37      4
13      4
27      4
94      4
39      4
31      4
91      4
41      4
15      3
       ..
74      2
100     2
33      2
45      2
56      2
99      2
12      1
48      1
66      1
8       1
47      1
93      1
51      1
87      1
20      1
85      1
60      1
19      1
67      1
80      1
24      1
26      1
77      1
30      1
32      1
34      1
35      1
63      1
69      1
78      1
Name: external_patient_id, dtype: int64


NameError: name 'ad' is not defined

QUESTIONS
1. When presented with a new dataset or database, what steps do you generally take
to evaluate it prior to working with it?
2. Based on the information provided above and the attached dataset, what 3
questions would you like to understand prior to conducting any analysis of the
data?
3. How would you prep the dataset provided for analysis?  Please list steps taken and
provide code used to prep the tables for analysis.
Using the datasets you have prepped, please answer the following questions and provide
all code/output used. Be sure to include and explain any assumptions made when
answering each question.
 
1. What is the average time elapsed between a patient’s initial diagnosis date and a
patient’s first treatment?  Does this time vary by gender?
2. How many patients are on nivolumab from 2012-2016?
3. Using the following risk stratification rules, please summarize the number of high,
medium, and low risk patients.

HighRisk       = Female; any­age; NON_WHITE       OR  Male; >= 70; NON_WHITE

MediumRisk = Female; >=75, WHITE  OR  Male; <70; NON_WHITE

LowRisk       = Female; < 75, WHITE  OR  Male; any­age; WHITE

4. Please create a visualization that could be used to help a medical researcher
understand how drug prevalence has changed over time.